# End-to-End Fine-Tuning Workflow in PostgreSQL

This notebook demonstrates how to fine-tune a GPT-2 model that lives *inside* PostgreSQL using the SQL helpers shipped with the `pg_llm` extension.
It walks through the entire lifecycle: importing weights, ingesting tokenizer assets, preparing a dataset, running training, and sampling
text from the tuned model. Each step is annotated so you can adapt it to your own data sources or infrastructure.

## 0. Prerequisites

Before running the notebook:

1. Build and install the extension following the instructions in the repository README.
2. Start a PostgreSQL instance with the extension available and set the `PGDSN` environment variable to your connection string (for example `postgresql://postgres@localhost:5432/postgres`).
3. Download a GPT-2 checkpoint and tokenizer files from Hugging Face (`vocab.json`, `merges.txt`).
4. Place any fine-tuning text files you want to ingest into a directory such as `./corpus`.

In [ ]:
import os
import pathlib
import textwrap

import psycopg

DSN = os.environ.get("PGDSN", "postgresql://postgres@localhost:5432/postgres")
MODEL_NAME = "gpt2-small"
WEIGHTS_ARCHIVE = pathlib.Path("/mnt/models/gpt2-small.npz")
TOKENIZER_VOCAB = pathlib.Path("./gpt2/vocab.json")
TOKENIZER_MERGES = pathlib.Path("./gpt2/merges.txt")
CORPUS_DIR = pathlib.Path("./corpus")
CORPUS_GLOB = str(CORPUS_DIR / "*.txt")

conn = psycopg.connect(DSN, autocommit=True)

def run_sql(sql, params=None):
    """Execute a SQL statement and pretty-print any returned rows."""
    params = params or ()
    with conn.cursor() as cur:
        cur.execute(sql, params)
        if cur.description:
            columns = [c.name for c in cur.description]
            rows = cur.fetchall()
            print(" | \t".join(columns))
            for row in rows:
                print(" | \t".join(str(v) for v in row))

run_sql("CREATE EXTENSION IF NOT EXISTS pg_llm")


## 1. Import Pretrained Weights

The helper function `pg_llm_import_npz` loads a converted GPT-2 checkpoint and splits it into the tensor layout expected by the extension.
Use the `scripts/convert_gpt2_checkpoint.py` helper to generate the NPZ archive beforehand.

In [ ]:
run_sql(
    "SELECT pg_llm_import_npz(%s, %s)",
    params=(str(WEIGHTS_ARCHIVE), MODEL_NAME),
)


## 2. Ingest Tokenizer Assets

Tokenizer merges and vocabulary entries live in dedicated tables. The `ingest_tokenizer.py` script loads them for you, but you can also use SQL
if the files are already accessible from the database server. Here we call the helper script via a notebook shell cell for reproducibility.

In [ ]:
!python scripts/ingest_tokenizer.py \
    --dsn {DSN} \
    --model {MODEL_NAME} \
    --vocab {TOKENIZER_VOCAB} \
    --merges {TOKENIZER_MERGES} \
    --truncate


## 3. Prepare a Tokenized Dataset

The dataset loader converts raw UTF-8 text into `(tokens, target)` arrays that match the model's context window. Each row represents a single training example.

In [ ]:
!python scripts/prepare_dataset.py \
    --dsn {DSN} \
    --tokenizer {MODEL_NAME} \
    --input {CORPUS_GLOB} \
    --block-size 1024 \
    --batch-size 512 \
    --truncate


After ingestion you can inspect the dataset distribution directly from SQL.

In [ ]:
run_sql(
    textwrap.dedent("""
    SELECT COUNT(*) AS examples, AVG(array_length(tokens, 1)) AS avg_tokens
      FROM llm_dataset
"""),
)


## 4. Launch Fine-Tuning

`llm_train` runs forward and backward passes entirely in SQL. Configure the scheduler, optimizer, and dropout parameters to match your experiment.
For a warm-up schedule you can reuse the same defaults that `llm_train_e2e.sql` demonstrates.

In [ ]:
run_sql(
    textwrap.dedent("""
    SELECT llm_train(%s, 1000, 12, 12, 768, 50257,
                   dropout_p => 0.1,
                   beta1 => 0.9,
                   beta2 => 0.999,
                   eps => 1e-8,
                   wd => 0.01,
                   lr_max => 2.5e-4,
                   warmup => 2000,
                   grad_clip => 1.0)
    """),
    params=(MODEL_NAME,),
)


### Monitor Training Progress

Training updates stream into `llm_train_log`. Query it to plot or log the loss curve.

In [ ]:
run_sql(
    textwrap.dedent("""
    SELECT step, loss
      FROM llm_train_log
     WHERE model = %s
  ORDER BY step DESC
     LIMIT 10
    """),
    params=(MODEL_NAME,),
)


## 5. Generate Text from the Tuned Model

The sampling helpers work on the fine-tuned parameters immediately. Use `llm_sampling.sql` as a reference for nucleus or top-k sampling strategies.

In [ ]:
prompt = "PostgreSQL models can"
run_sql(
    textwrap.dedent("""
    SELECT llm_generate(%s, 64, 0.8, 40, 0.95)
    """),
    params=(prompt,),
)


## 6. Clean Up (Optional)

Drop temporary artifacts after experimentation to reclaim storage.

In [ ]:
run_sql(
    textwrap.dedent("""
    DELETE FROM llm_train_log WHERE model = %s;
    DELETE FROM llm_dataset;
    DELETE FROM llm_param WHERE model = %s;
    """),
    params=(MODEL_NAME, MODEL_NAME),
)


---

### Troubleshooting

* If `pg_llm_import_npz` fails, ensure the path is readable by the PostgreSQL server process.
* The helper scripts assume Hugging Face checkpoints; custom models need matching tensor names.
* Use `EXPLAIN (ANALYZE, BUFFERS)` around training queries to profile performance bottlenecks.
* Consider using [pgvector](https://github.com/pgvector/pgvector) or similar extensions if you want hybrid retrieval + generation workflows.